In [3]:
import pandas as pd
import numpy as np

from  sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline

from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

In [4]:
all_data_df = pd.read_csv('all_data_df.csv')
all_data_df.index = all_data_df['SEQN']
all_data_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 29902 entries, 62161.0 to 93698.0
Columns: 674 entries, SEQN to WHQ150
dtypes: float64(672), object(2)
memory usage: 154.0+ MB


# Create Label

In [5]:
# Select mental health screener columns and drop data that are empty
mental_health_df = all_data_df.loc[:, 'DPQ010':'DPQ100'].dropna(how='all')
all_data_df = all_data_df.loc[mental_health_df.index]

In [6]:
def mh(x):
    if x == '\.':
        return 'missing'
    elif x == 1:
        return 'several days'
    elif x == 2:
        return 'more than half the days'
    elif x == 3:
        return 'nearly every day'
    elif x == 7:
        return 'refused'
    elif x == 9:
        return "don't know"
    else:
        return 'not at all'

for col in mental_health_df.columns:
    mental_health_df[col] = mental_health_df[col].apply(lambda x: mh(x))

mental_health_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 15513 entries, 62161.0 to 93702.0
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   DPQ010  15513 non-null  object
 1   DPQ020  15513 non-null  object
 2   DPQ030  15513 non-null  object
 3   DPQ040  15513 non-null  object
 4   DPQ050  15513 non-null  object
 5   DPQ060  15513 non-null  object
 6   DPQ070  15513 non-null  object
 7   DPQ080  15513 non-null  object
 8   DPQ090  15513 non-null  object
 9   DPQ100  15513 non-null  object
dtypes: object(10)
memory usage: 1.3+ MB


In [7]:
def calc(row):
    sum = 0
    for i in ['DPQ010', 'DPQ020', 'DPQ030', 'DPQ040', 
              'DPQ050', 'DPQ060', 'DPQ070','DPQ080', 
              'DPQ090', 'DPQ100']:
        if row[i] == 'several days':
            sum += 1
        if row[i] == 'more than half the days':
            sum += 2
        if row[i] == 'nearly every day':
            sum += 3
    return sum

In [8]:
# Use 10 as threshold for depression
mental_health_df['labels_raw'] = mental_health_df.apply(calc, axis=1)
mental_health_df['labels'] = mental_health_df['labels_raw'].apply(lambda x: 1 if x >= 10 else 0)

# Select Features

In [30]:
features = [
    #diabetes
    'DIQ170','DIQ175A','DIQ175B','DIQ175C',
    'DIQ175D','DIQ175E','DIQ175F','DIQ175G','DIQ175H','DIQ175I','DIQ175J','DIQ175K',
    'DIQ175L', 'DIQ175M','DIQ175N','DIQ175O','DIQ175P','DIQ175Q','DIQ175R','DIQ175S',
    'DIQ175T','DIQ175U','DIQ175V','DIQ175W','DIQ060U',
    'DIQ260U',
    #sleep disorder
    #physical activity
    'PAD680','PAQ706',
    #weight history
    'WHD080A','WHD080B',
    'WHD080C','WHD080D','WHD080E','WHD080F','WHD080G','WHD080H','WHD080I','WHD080J',
    'WHD080K','WHD080M','WHD080N','WHD080O','WHD080P','WHD080Q','WHD080R','WHD080S',
    'WHD080T','WHD080L',
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E',
    'ECQ150', 
    #alcohol issues
    #early childhood
    'ECD010','ECQ020','ECD070A','ECD070B','ECQ080','ECQ090','WHQ030E','MCQ080E','ECQ150', 
    #hospital access
    'HUQ071','HUQ090', 
    #health status
    'HSQ520','HSAQUEX',  
    #income
    'INQ150',
    #housing
    #occupation
    #diet nutrition
    'DBQ010','DBD030','DBD041','DBD050','DBD055','DBD061','DBQ073A','DBQ073B','DBQ073C',
    'DBQ073D','DBQ073E','DBQ073U','DBQ700','DBQ223A','DBQ223B','DBQ223C',
    'DBQ223D','DBQ223E','DBQ223U',
    'DBQ330','DBD381','DBQ390','DBQ400','DBD411','DBQ421','DBQ424',
    #drug use
    'DUQ250','DUQ280','DUQ300','DUQ310Q','DUQ310U','DUQ320',
    'DUQ360','DUQ380A','DUQ380B','DUQ380C','DUQ380D','DUQ380E',
    'DUQ400U'
]

final_features = []

for feature in features:
    if feature not in final_features and feature != 'SEQN':
        final_features.append(feature)

# Split Data

https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html

In [12]:
X = all_data_df[final_features]
y = mental_health_df['labels']

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42)

# Train Model

https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html

Dummy Classifier

In [14]:
dummy_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', DummyClassifier(strategy='stratified', random_state = 42))
])
dummy_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf',
                 DummyClassifier(random_state=42, strategy='stratified'))])

Linear Regression

In [15]:
lr_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('clf', LogisticRegression())
])
lr_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('clf', LogisticRegression())])

Linear Regression W/PCA

In [16]:
lr_pca_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', LogisticRegression())
])
lr_pca_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)), ('clf', LogisticRegression())])

Linear Discriminant Analysis

In [17]:
lda_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', LinearDiscriminantAnalysis())
])
lda_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf', LinearDiscriminantAnalysis())])

Decision Tree Classifier

In [18]:
dt_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', DecisionTreeClassifier(random_state = 42))
])
dt_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf', DecisionTreeClassifier(random_state=42))])

Gradient Boosting Classifier

In [19]:
gdclass_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
                                       max_depth=1, random_state=42))
])
gdclass_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf',
                 GradientBoostingClassifier(learning_rate=1.0, max_depth=1,
                                            random_state=42))])

Random Forest Classifier

In [20]:
rfclass_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', RandomForestClassifier(max_depth=2, random_state=42))
])
rfclass_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf', RandomForestClassifier(max_depth=2, random_state=42))])

Gaussian Naive Bayes

In [21]:
gnb_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', GaussianNB())
])
gnb_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)), ('clf', GaussianNB())])

Quadratic Discriminant Analysis

In [22]:
qda_pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('enc', OneHotEncoder(sparse=False, handle_unknown='ignore')), 
    ('red', PCA(n_components=10)),
    ('clf', QuadraticDiscriminantAnalysis())
])
qda_pipe.fit(X_train, y_train)

Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')),
                ('enc', OneHotEncoder(handle_unknown='ignore', sparse=False)),
                ('red', PCA(n_components=10)),
                ('clf', QuadraticDiscriminantAnalysis())])

# Model Evaluation

In [23]:
dummy_training_score = roc_auc_score(y_train.values, dummy_pipe.predict_proba(X_train)[:, 1])
dummy_validation_score = roc_auc_score(y_val.values, dummy_pipe.predict_proba(X_val)[:, 1])

lr_training_score = roc_auc_score(y_train.values, lr_pipe.predict_proba(X_train)[:, 1])
lr_validation_score = roc_auc_score(y_val.values, lr_pipe.predict_proba(X_val)[:, 1])

lr_pca_training_score = roc_auc_score(y_train.values, lr_pca_pipe.predict_proba(X_train)[:, 1])
lr_pca_validation_score = roc_auc_score(y_val.values, lr_pca_pipe.predict_proba(X_val)[:, 1])

lda_training_score = roc_auc_score(y_train.values, lda_pipe.predict_proba(X_train)[:, 1])
lda_validation_score = roc_auc_score(y_val.values, lda_pipe.predict_proba(X_val)[:, 1])

dt_training_score = roc_auc_score(y_train.values, dt_pipe.predict_proba(X_train)[:,1])
dt_validation_score = roc_auc_score(y_val.values, dt_pipe.predict_proba(X_val)[:,1])

gdclass_training_score = roc_auc_score(y_train.values, gdclass_pipe.predict_proba(X_train)[:,1])
gdclass_validation_score = roc_auc_score(y_val.values, gdclass_pipe.predict_proba(X_val)[:,1])

rfclass_training_score = roc_auc_score(y_train.values, rfclass_pipe.predict_proba(X_train)[:,1])
rfclass_validation_score = roc_auc_score(y_val.values, rfclass_pipe.predict_proba(X_val)[:,1])

gnb_training_score = roc_auc_score(y_train.values, gnb_pipe.predict_proba(X_train)[:,1])
gnb_validation_score = roc_auc_score(y_val.values, gnb_pipe.predict_proba(X_val)[:,1])

qda_training_score = roc_auc_score(y_train.values, qda_pipe.predict_proba(X_train)[:,1])
qda_validation_score = roc_auc_score(y_val.values, qda_pipe.predict_proba(X_val)[:,1])

In [24]:
pd.DataFrame.from_dict(
    {
        'dummy': {
            'training score': dummy_training_score,
            'validation score': dummy_validation_score
        },
        'logistic regression': {
            'training score': lr_training_score,
            'validation score': lr_validation_score
        },
        'logistic regression pca': {
            'training score': lr_pca_training_score,
            'validation score': lr_pca_validation_score
        },
        'linear discriminant analysis': {
            'training score': lda_training_score,
            'validation score': lda_validation_score
        },
        'decision tree classifier': {
            'training score': dt_training_score,
            'validation score': dt_validation_score
        },
        'gradient boosting classifier': {
            'training score': gdclass_training_score,
            'validation score': gdclass_validation_score
        },
        'random forest classifier': {
            'training score': rfclass_training_score,
            'validation score': rfclass_validation_score
        },
        'gaussian naive bayes': {
            'training score': gnb_training_score,
            'validation score': gnb_validation_score
        },
        'quadratic discriminant analysis': {
            'training score': qda_training_score,
            'validation score': qda_validation_score
        }
    
    }, 
    orient='index')

,training score,validation score
dummy,0.492412,0.488668
logistic regression,0.748345,0.726603
logistic regression pca,0.709406,0.679108
linear discriminant analysis,0.709764,0.684005
decision tree classifier,0.861990,0.562292
gradient boosting classifier,0.747171,0.694985
random forest classifier,0.702317,0.668296
gaussian naive bayes,0.686763,0.669262
quadratic discriminant analysis,0.707930,0.675564


# Feature Importance

https://scikit-learn.org/stable/modules/permutation_importance.html

In [31]:
r = permutation_importance(
    lr_pca_pipe, 
    X_val, 
    y_val,
    n_repeats=10,
    n_jobs=-1,
    random_state=42
)

In [32]:
feature_importances = pd.DataFrame.from_dict(
    {
        'importance_means': r['importances_mean'],
        'importances_std': r['importances_std']
    }, orient='columns'
)
feature_importances.index = X_val.columns

In [33]:
feature_importances.sort_values('importance_means', ascending = False)

,importance_means,importances_std
HUQ090,0.004062,0.001472
HUQ071,0.002579,0.000912
DBQ700,0.002450,0.001071
PAD680,0.000903,0.001124
INQ150,0.000580,0.000347
...,...,...
WHD080A,0.000000,0.000000
PAQ706,0.000000,0.000000
DUQ400U,0.000000,0.000000
DUQ290,-0.000064,0.000193


# Hyperparameter Tuning

- https://machinelearningmastery.com/hyperparameters-for-classification-machine-learning-algorithms/
- https://medium.com/@kocur4d/hyper-parameter-tuning-with-pipelines-5310aff069d6

In [34]:
grid_params = {
  'clf__solver': ['newton-cg', 'lbfgs'],
  'clf__penalty': ['none', 'l1'],  
  'clf__C': [100, 10],  
}

In [35]:
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=2, random_state=42)
grid_search = GridSearchCV(
    estimator=lr_pca_pipe, 
    param_grid=grid_params, 
    n_jobs=-1, 
    cv=cv, 
    scoring='roc_auc',
    error_score=0
)
grid_result = grid_search.fit(X_train, y_train)


In [36]:
grid_result.best_score_

0.7059279521921007

In [37]:
grid_result.best_params_

{'clf__C': 10, 'clf__penalty': 'none', 'clf__solver': 'lbfgs'}

In [38]:
grid_result.predict(X_val)

array([0, 0, 0, ..., 0, 0, 0])